In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load and preprocess the dataset (use FER-2013 or similar facial expression datasets)
def load_dataset():
    # Assuming FER-2013 dataset or similar format with "pixels" and "emotion" columns
    import pandas as pd
    data = pd.read_csv(r'C:\Users\lokes\Downloads\fer2013.csv')
    X = []
    y = []
    for i in range(len(data)):
        pixels = np.array(data['pixels'][i].split(), dtype='float32').reshape(48, 48)
        X.append(pixels)
        y.append(data['emotion'][i])
    X = np.array(X)
    y = np.array(y)
    X = X / 255.0  # Normalize pixel values
    y = to_categorical(y)  # One-hot encode emotion labels
    return X, y




In [ ]:

# Build the CNN model
def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')  # Assuming 7 emotion classes
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_model((48, 48, 1))
    model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))
    return model, X_test, y_test

# Real-time emotion detection with webcam
def emotion_detection(model):
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        detected_faces = faces.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in detected_faces:
            roi = gray[y:y+h, x:x+w]
            roi = cv2.resize(roi, (48, 48))
            roi = roi.reshape(1, 48, 48, 1) / 255.0
            prediction = model.predict(roi)
            emotion = emotion_labels[np.argmax(prediction)]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function
if __name__ == "__main__":
    X, y = load_dataset()
    X = X.reshape(-1, 48, 48, 1)  # Reshape for CNN
    model, X_test, y_test = train_model(X, y)
    print("Model trained successfully. Starting real-time emotion detection.")
    emotion_detection(model)


c:\Users\lokes\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.2618 - loss: 1.8005 - val_accuracy: 0.3975 - val_loss: 1.5840
Epoch 2/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.3800 - loss: 1.5919 - val_accuracy: 0.4398 - val_loss: 1.4584
Epoch 3/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.4217 - loss: 1.4967 - val_accuracy: 0.4634 - val_loss: 1.3976
Epoch 4/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.4488 - loss: 1.4346 - val_accuracy: 0.4826 - val_loss: 1.3594
Epoch 5/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.4600 - loss: 1.3948 - val_accuracy: 0.4935 - val_loss: 1.3288
Epoch 6/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.4718 - loss: 1.3596 - val_accuracy: 0.5046 - val_loss: 1.3005
Epoch 7/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.4896 - loss: 1.3288 - val_accuracy: 0.5095 - val_loss: 1.2868
Epoch 8/15
449/449 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.4988 - loss: 1.3080 - 